# Attibutes

category
crawl_date
sub_cat
title
mrp
urlh
http_status
pack_size
available_price


## 1. Number of URLH which are overlapping (Common) in two files.

In [1]:
import json
urlh_tod={0} #empty sets to store urlh data from today file
urlh_yest={1} #empty sets to store urlh data from yesterday file
with open("today.json") as file:
    for obj in file: #read each json obj and store in a temp dict
        dic = json.loads(obj)
        urlh_tod.add(dic["urlh"])
with open("yesterday.json") as file2:
    for obj in file2:
        dic = json.loads(obj)
        urlh_yest.add(dic["urlh"])
y = urlh_tod.intersection(urlh_yest)# Built-in set function to find the intersection of 2 sets(toay,yesterday)
print(len(y)) #The final common elements count


129673


### 2. For all the URLH which are overlapping, calculate the price difference (wrt available_price) if there is any between yesterday's and today's crawls (scraped data). There might be duplicate URLHs in which case you can choose the first valid (with http_status 200) record. (todo)

In [ ]:
tod=[] #empty lists to store urlh data from today file
yest=[] #empty lists to store urlh data from yesterday file
with open("today.json") as file:
    for obj in file: #read each json obj and store in a temp dict
        dic = json.loads(obj)
        tod.append(dic)
with open("yesterday.json") as file2:
    for obj in file2:
        dic = json.loads(obj)
        yest.append(dic)    
        
main_yest = {}
main_tod = {}
for i in yest:
    if i["urlh"] in main_yest.keys():
        if (main_yest[i["urlh"]][0] != 200) and (i["http_status"]==200) and (i["available_price"] != None):
            main_yest[i["urlh"]] = [i["http_status"],i["available_price"]]
    elif i["available_price"] != None:
        temp={i["urlh"]:[i["http_status"],i["available_price"]]}
        main_yest.update(temp)
for i in tod:
    if i["urlh"] in main_tod.keys():
        if (main_tod[i["urlh"]][0] != 200) and (i["http_status"]==200) and (i["available_price"] != None):
            main_tod[i["urlh"]] = [i["http_status"],i["available_price"]]
    elif i["available_price"] != None:
        temp={i["urlh"]:[i["http_status"],i["available_price"]]}
        main_tod.update(temp)

unlist = list(main_yest.keys())
diff = {}
for i in unlist:
    if i in main_tod.keys():
        diffval = float(main_tod[i][1])-float(main_yest[i][1])
        temp = {i:[main_tod[i][0],diffval]}
        diff.update(temp)
for i in diff.items():
    print(i)

# 3. Number of Unique categories in both files.

In [ ]:
cat_today={0} #empty sets to store category data from today file
cat_yest={1} #empty sets to store category data from yesterday file
with open("today.json") as file:
    for obj in file: #read each json obj and store in a temp dict
        dic = json.loads(obj)
        cat_today.add(dic["category"])
with open("yesterday.json") as file2:
    for obj in file2:
        dic = json.loads(obj)
        cat_yest.add(dic["category"])
y = cat_today.union(cat_yest)#the built-in union function of set that concate the both sets and avoids the duplications
y.remove(0) #as this was a junk value just to define a set 
y.remove(1) #we will remove it
print(len(y))

## 4. Display List of categories which is not overlapping (Common) from two given files.

In [ ]:
cat_today=[] #empty list to store category data from today file
cat_yest=[]  #empty list to store category data from yesterday file
with open("today.json") as file:
    for obj in file:
        dic = json.loads(obj)
        cat_today.append(dic["category"])
with open("yesterday.json") as file2:
    for obj in file2:
        dic = json.loads(obj)
        cat_yest.append(dic["category"])
dicory={} # empty dictionary that will store the non common category 
for i in cat_today:
    temp={i:0}
    dicory.update(temp) #take the category from the first file and assign them as keys
for i in cat_yest:
    if i in dicory.keys():# check the category of yesterday file in dictionary keys
        dicory.pop(i) # pop the common category 
print(dicory)


#alternative way
#y = c.symmetric_difference(d)
#print(y)

## 5. Generate the stats with count of urlh for all taxonomies (taxonomy is concatenation of category and subcategory separated by " > ") for today's file.

In [ ]:
today_data = [] 
cat_today  = []
count=0
with open("today.json") as file:
    for obj in file: 
        dic = json.loads(obj)
        cat_today.append(dic["category"]) #store the category in list
        today_data.append(dic) # store the entire object in list
cat_today = list(set(cat_today)) #removes the duplication of a list
for i in cat_today:
    sub_catlist = []
    for c_obj in today_data:
        if c_obj["category"] == i:
            sub_catlist.append(c_obj["subcategory"])
    uniq_list = list(set(sub_catlist))
    
    for j in uniq_list:
        count+=1
        print(f"{count} {i} > {j} ==> {sub_catlist.count(j)}")

##  6. Generate a new file where mrp is normalized. If there is a 0 or a non-float value or the key doesn't exist, make it "NA".

In [ ]:
t_data=[]
count=0
with open("today.json") as file:
    for obj in file: 
        dic = json.loads(obj)
        t_data.append(dic)
for i in t_data:
    if i["mrp"] == None or float(i["mrp"])<=0 or type(i['mrp'])==int :
        i["mrp"]="NA"
with open("newdata.json","w") as nd:
    for i in t_data:
        json.dump(i,nd,indent=4)

# 7. Display the title and price of 10 items having least price.

In [ ]:
data = []
with open("today.json") as file:
    for obj in file: #read each json obj and store in a temp dict
        dic = json.loads(obj)
        if dic['title'] != None or dic['available_price'] != None:
            temp = [dic["available_price"],dic["title"]]
            data.append(temp)
data = sorted(data)
for i in range(10):
    print(f"{data[i][1]} --> {data[i][0]}")

# 8. Display the top 5 subcategory having highest items.

In [ ]:
subcat = []
data=[]
with open("today.json") as fo:
    for o in fo:
        dic = json.loads(o)
        subcat.append(dic["subcategory"])
unisub = list(set(subcat))

for i in unisub:
    temp = [subcat.count(i),i]
    data.append(temp)

data = sorted(data,reverse=True)
for i in range(5):
    print(f"{data[i][1]}-->{data[i][0]}")

## 9. Display stats of how many items have failed status (http_status other than 200 is to be considered as failure).

In [ ]:
statusdata = []
with open("today.json") as f:
    for i in f:
        d=json.loads(i)
        if d["http_status"] > "200":
            statusdata.append(d["http_status"])
uq = set(statusdata)            
print("Http_status --> count")
for i in uq:
    print(f"{i} --> {statusdata.count(i)}")